## GAT

In [1]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf
import torch

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [2]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))

In [3]:
max_Id = data["Id"].unique().max()

In [4]:
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)

In [5]:
data = pd.concat((data, gw_data))
data = cudf.from_pandas(data)

In [6]:
data = data.sort_values("Id")

In [7]:
unique_ids = data["Id"].unique()

In [8]:
data = data.reset_index(drop=True)

In [9]:
data.pop("index")

0          665587
1          665588
2          665589
3          665590
4          665591
            ...  
1819430    665584
1819431    665581
1819432    665583
1819433    665586
1819434    665585
Name: index, Length: 1819435, dtype: int64

In [10]:
data = data.sort_values(["Id", "Time"], ascending=[True, False])

## Creating edge index

In [12]:
import numpy as np
import cupy as cp
import time

source_edge_index = np.array([], dtype= cp.int32)
target_edge_index = np.array([], dtype= cp.int32)

start = time.time()
j = 0
for Id, group in data.groupby("Id"):
    indices = group.index
    offset = indices[0]
    num_nodes = len(indices)
    edge_index = torch.zeros((2, sum(range(num_nodes + 1))), dtype=torch.long)+offset

    ## Self edges
    edge_index[:, 0:num_nodes] = (torch.arange(num_nodes) + offset).view(1, -1)
    idx = num_nodes
    for i in range(1, num_nodes):
        edge_index[1, idx:idx + i] = i+offset
        edge_index[0, idx:idx + i] = torch.arange(i)+offset
        idx += i
    
    source_edge_index = np.concatenate((source_edge_index, edge_index[0, :].numpy()))
    target_edge_index = np.concatenate((target_edge_index, edge_index[1, :].numpy()))
    j+=1
    if j % 1000 == 0:
        print(f"{str(j / unique_ids.shape[0] * 100)} %")

print(time.time() - start )
print(source_edge_index.shape)
print(target_edge_index.shape)

edge_index = np.asarray([np.asarray(source_edge_index), np.asarray(target_edge_index)])
edge_index = torch.tensor(edge_index)
pd.DataFrame(edge_index.numpy().transpose()).to_csv("dir_edge_index_sorted_ids_w_gw_corr.csv", index=False)

0.11540454486178575 %
0.2308090897235715 %
0.34621363458535725 %
0.461618179447143 %
0.5770227243089288 %
0.6924272691707145 %
0.8078318140325003 %
0.923236358894286 %
1.0386409037560718 %
1.1540454486178575 %
1.2694499934796433 %
1.384854538341429 %
1.5002590832032148 %
1.6156636280650005 %
1.7310681729267863 %
1.846472717788572 %
1.9618772626503578 %
2.0772818075121435 %
2.1926863523739293 %
2.308090897235715 %
2.423495442097501 %
2.5388999869592865 %
2.6543045318210723 %
2.769709076682858 %
2.885113621544644 %
3.0005181664064295 %
3.1159227112682153 %
3.231327256130001 %
3.346731800991787 %
3.4621363458535725 %
3.5775408907153583 %
3.692945435577144 %
3.80834998043893 %
3.9237545253007156 %
4.039159070162501 %
4.154563615024287 %
4.269968159886073 %
4.385372704747859 %
4.500777249609644 %
4.61618179447143 %
4.731586339333216 %
4.846990884195002 %
4.962395429056787 %
5.077799973918573 %
5.193204518780359 %
5.308609063642145 %
5.42401360850393 %
5.539418153365716 %
5.654822698227502 %

46.508031579299654 %
46.623436124161444 %
46.73884066902323 %
46.854245213885015 %
46.9696497587468 %
47.08505430360859 %
47.200458848470376 %
47.31586339333216 %
47.43126793819394 %
47.54667248305573 %
47.66207702791752 %
47.7774815727793 %
47.89288611764108 %
48.00829066250287 %
48.12369520736466 %
48.239099752226444 %
48.354504297088226 %
48.469908841950016 %
48.585313386811805 %
48.70071793167359 %
48.81612247653537 %
48.93152702139716 %
49.04693156625895 %
49.16233611112073 %
49.27774065598251 %
49.3931452008443 %
49.50854974570609 %
49.62395429056787 %
49.739358835429655 %
49.854763380291445 %
49.970167925153234 %
50.085572470015016 %
50.2009770148768 %
50.31638155973859 %
50.43178610460038 %
50.54719064946216 %
50.66259519432394 %
50.77799973918573 %
50.89340428404752 %
51.0088088289093 %
51.124213373771084 %
51.239617918632874 %
51.35502246349466 %
51.470427008356445 %
51.58583155321823 %
51.70123609808002 %
51.816640642941806 %
51.93204518780359 %
52.04744973266537 %
52.162854

93.82389497263182 %
93.9392995174936 %
94.05470406235538 %
94.17010860721717 %
94.28551315207896 %
94.40091769694075 %
94.51632224180253 %
94.63172678666432 %
94.7471313315261 %
94.86253587638788 %
94.97794042124967 %
95.09334496611146 %
95.20874951097325 %
95.32415405583504 %
95.43955860069681 %
95.5549631455586 %
95.67036769042039 %
95.78577223528217 %
95.90117678014396 %
96.01658132500575 %
96.13198586986753 %
96.24739041472932 %
96.3627949595911 %
96.47819950445289 %
96.59360404931468 %
96.70900859417645 %
96.82441313903824 %
96.93981768390003 %
97.05522222876182 %
97.17062677362361 %
97.28603131848539 %
97.40143586334717 %
97.51684040820896 %
97.63224495307074 %
97.74764949793253 %
97.86305404279432 %
97.9784585876561 %
98.0938631325179 %
98.20926767737967 %
98.32467222224146 %
98.44007676710325 %
98.55548131196502 %
98.67088585682681 %
98.7862904016886 %
98.90169494655039 %
99.01709949141218 %
99.13250403627396 %
99.24790858113575 %
99.36331312599754 %
99.47871767085931 %
99.5941

In [11]:
import torch

In [12]:
import torch
edge_index = torch.tensor(pd.read_csv("dir_edge_index_sorted_ids_w_gw_corr.csv", header=None, skiprows=1).values.transpose(), dtype=torch.long)

In [20]:
data = data.reset_index(drop=True)

In [23]:
data[data["Id"] == 8]

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label
7,8,64,W,Control,Leipzig,Training,AMB,1,3884100.0,<NA>,<NA>,1.30,5.2,102.3,<NA>,208.0,2.56,5.5,Control
8,8,64,W,Control,Leipzig,Training,AMB,1,3277380.0,<NA>,<NA>,5.87,5.3,103.0,<NA>,194.0,2.71,7.2,Control
9,8,64,W,Control,Leipzig,Training,AMB,1,2766900.0,<NA>,<NA>,14.15,5.6,99.6,<NA>,230.0,2.82,10.4,Control
10,8,64,W,Control,Leipzig,Training,AMB,1,2073600.0,<NA>,<NA>,2.94,5.1,100.0,<NA>,346.0,2.62,5.6,Control
11,8,64,W,Control,Leipzig,Training,AMB,1,1474140.0,<NA>,<NA>,27.37,5.5,102.5,<NA>,413.0,2.77,6.4,Control
12,8,64,W,Control,Leipzig,Training,AMB,1,0.0,<NA>,<NA>,37.43,4.2,103.3,<NA>,329.0,2.11,4.3,Control


In [26]:
edge_index[:, 10:20]

tensor([[ 9, 10, 11, 12,  7,  7,  8,  7,  8,  9],
        [ 9, 10, 11, 12,  8,  9,  9, 10, 10, 10]])

In [27]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES
data[SEX_CATEGORY_COLUMN_NAME] = data.loc[:, SEX_COLUMN_NAME] =="W"

In [28]:
data[SEX_CATEGORY_COLUMN_NAME] = data[SEX_CATEGORY_COLUMN_NAME].astype("int8")
data["Label"] = data["Label"] == "Sepsis"
data["Label"] = data["Label"].astype("int8")

In [29]:
import numpy as np
import matplotlib.pyplot as plt

def getPositionEncoding(seq_len, d = len(FEATURES), n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P

In [30]:
X_features = torch.tensor(data[FEATURES].values.get()).type(torch.float32)
y = torch.tensor(data["Label"].values.get()).type(torch.long)

In [31]:
from dataAnalysis.FeatureImportance import normalize

X_features = normalize(X_features)

## Creating positional encodings

In [ ]:
import numpy as np
import time

pos_encodings = None

start = time.time()
i = 0
final_shape = 0
for Id, group in data.groupby("Id"):
    encoding = getPositionEncoding(group.shape[0])
#     sorted_group = group.reset_index().sort_values("Time")
#     encoding = encoding[sorted_group.index.to_numpy()]
    final_shape += group.shape[0]
    pos_encodings = encoding if pos_encodings is None else np.concatenate((pos_encodings, encoding), axis=0)
    i+=1
    if i % 1000 == 0:
        print(f"{str(i / unique_ids.shape[0] * 100)} %")

print(pos_encodings.shape[0] == final_shape)
pd.DataFrame(pos_encodings).to_csv("pos_encodings_new_w_gw_corr.csv", index=False)

0.11540454486178575 %
0.2308090897235715 %
0.34621363458535725 %
0.461618179447143 %
0.5770227243089288 %
0.6924272691707145 %
0.8078318140325003 %
0.923236358894286 %
1.0386409037560718 %
1.1540454486178575 %
1.2694499934796433 %
1.384854538341429 %
1.5002590832032148 %
1.6156636280650005 %
1.7310681729267863 %
1.846472717788572 %
1.9618772626503578 %
2.0772818075121435 %
2.1926863523739293 %
2.308090897235715 %
2.423495442097501 %
2.5388999869592865 %
2.6543045318210723 %
2.769709076682858 %
2.885113621544644 %
3.0005181664064295 %
3.1159227112682153 %
3.231327256130001 %
3.346731800991787 %
3.4621363458535725 %
3.5775408907153583 %
3.692945435577144 %
3.80834998043893 %
3.9237545253007156 %
4.039159070162501 %
4.154563615024287 %
4.269968159886073 %
4.385372704747859 %
4.500777249609644 %
4.61618179447143 %
4.731586339333216 %
4.846990884195002 %
4.962395429056787 %
5.077799973918573 %
5.193204518780359 %
5.308609063642145 %
5.42401360850393 %
5.539418153365716 %
5.654822698227502 %

46.508031579299654 %
46.623436124161444 %
46.73884066902323 %
46.854245213885015 %
46.9696497587468 %
47.08505430360859 %
47.200458848470376 %
47.31586339333216 %
47.43126793819394 %
47.54667248305573 %
47.66207702791752 %
47.7774815727793 %
47.89288611764108 %
48.00829066250287 %
48.12369520736466 %
48.239099752226444 %
48.354504297088226 %
48.469908841950016 %
48.585313386811805 %
48.70071793167359 %
48.81612247653537 %
48.93152702139716 %
49.04693156625895 %
49.16233611112073 %
49.27774065598251 %
49.3931452008443 %
49.50854974570609 %
49.62395429056787 %
49.739358835429655 %
49.854763380291445 %
49.970167925153234 %
50.085572470015016 %
50.2009770148768 %
50.31638155973859 %
50.43178610460038 %
50.54719064946216 %
50.66259519432394 %
50.77799973918573 %
50.89340428404752 %
51.0088088289093 %
51.124213373771084 %
51.239617918632874 %
51.35502246349466 %
51.470427008356445 %
51.58583155321823 %
51.70123609808002 %
51.816640642941806 %
51.93204518780359 %
52.04744973266537 %
52.162854

In [ ]:
pos_encodings = pd.read_csv("pos_encodings_new_w_gw_corr.csv", header=None, skiprows=1)

In [ ]:
pos_encodings = pos_encodings.values

In [ ]:
X_features

In [ ]:
X_new_features = X_features + pos_encodings

In [ ]:
X_new_features.shape

In [ ]:
def ratio_bool_switch(tensor, ratio = 0.8):
    random = np.random.uniform(0, 1 ,tensor.shape[0])
    val_ratio_mask = (random >= ratio)
    train_ratio_mask = (random < ratio)
    val_mask = np.logical_and(tensor.tolist(), val_ratio_mask.tolist())
    train_mask = np.logical_and(tensor.tolist(), train_ratio_mask.tolist())
    return torch.from_numpy(train_mask).type(torch.bool), torch.from_numpy(val_mask).type(torch.bool)

In [ ]:
from torch_geometric.data import Data

train_mask_ser = data["Set"] != "Validation"
train_mask, val_mask = ratio_bool_switch(train_mask_ser.values)
test_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values.get(), (data["Center"] == "Leipzig").values.get())).type(torch.bool)
test_gw_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values.get(), (data["Center"] == "Greifswald").values.get())).type(torch.bool)
graph = Data(x=X_new_features, train_mask = train_mask, test_mask=test_mask, val_mask=val_mask, y= y, edge_index=edge_index,
             test_gw_mask = test_gw_mask)

In [ ]:
test_gw_mask.sum() + test_mask.sum()+val_mask.sum()+train_mask.sum()

In [ ]:
graph.edge_index

In [ ]:
graph = graph.cpu()

In [ ]:
from torch_geometric.loader import NeighborLoader

kwargs = {
    "num_neighbors":[-1] * 2,
    "batch_size":50_000
}
loader = NeighborLoader(
    graph,
    input_nodes=graph.train_mask,
    **kwargs
)
val_loader = NeighborLoader(
    graph,
    input_nodes=graph.val_mask,
    **kwargs
)

In [ ]:
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

# graph = graph.to(device)
WEIGHT = torch.tensor([664*4/5])#
WEIGHT = WEIGHT.to(device)

print("Data shifted to the device " + str(device))

In [ ]:
graph.val_mask.sum()

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        HEADS = 5
        
        conv_1= GATConv(input_dim, hidden_dim,heads=HEADS, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end
        

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = x.type(torch.float)
        x = self.conv_1(x, edge_index) 
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x = self.conv_end(x, edge_index)
        return x
            
    def predict_proba(self, graph, mask):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(graph)
            scores = torch.sigmoid(torch.squeeze(logits[mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, graph, mask):
        return torch.round(self.predict_proba(graph, mask)[:, 1])

In [ ]:
import torch 
from tqdm.notebook import tqdm
from dataAnalysis.Metrics import Evaluation

class ModelWrapper():
    def __init__(self, graph):
        self.LEARNING_RATE = 3e-4
        self.MAX_EPOCHS = 10000 #40000

        self.model = GraphNeuralNetwork(hidden_dim = 128, out_channels=1) 
        self.model = self.model.to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.LEARNING_RATE,betas=(0.9, 0.999), eps=1e-08)
        self.graph = graph
        
        self.last_loss = 0
        self.increased_loss = 0
        self.BREAKING_THRESHOLD = 5  
        self.val_loss = []
        self.train_loss = []
        self.epochs = self.MAX_EPOCHS
        
    def change_lr(self, lr):
        for g in self.optimizer.param_groups:
            g['lr'] = lr
    
    def validate(self):
        with torch.inference_mode():
            self.model.eval()
            acc_loss = 0
            batch_size = 0
            for batch, graph in enumerate(val_loader):
                graph = graph.to(device)
                out = self.model(graph)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), graph.y.type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
            avg_loss = acc_loss / batch_size
            self.val_loss.append(avg_loss)
            if avg_loss > self.last_loss:
                self.increased_loss += 1
            else:
                self.increased_loss = 0
            self.last_loss = avg_loss

    def train(self):
        for epoch in tqdm(range(self.MAX_EPOCHS)):
            acc_loss = 0
            batch_size = 0
                
#             if epoch == 10:
#                 self.change_lr(1e-5)
#                 self.plot_loss()
#                 self.evaluate()
#             if epoch == 20:
#                 self.change_lr(5e-4)
#                 self.plot_loss()
#                 self.evaluate()
#             if epoch == 30:
#                 self.change_lr(1e-4)
#                 self.plot_loss()
#                 self.evaluate()
            for batch, graph in enumerate(loader):
                graph = graph.to(device)
                self.model.train()
                self.optimizer.zero_grad()
                out = self.model(graph)
                loss = F.binary_cross_entropy_with_logits(torch.squeeze(out), graph.y.type(torch.float32),
                                                          pos_weight=WEIGHT)
                acc_loss += loss.item()
                batch_size += 1
                loss.backward()
                self.optimizer.step()
            self.train_loss.append(acc_loss/batch_size)
            self.validate() 

            if self.increased_loss >= self.BREAKING_THRESHOLD:
                self.epochs = epoch + 1
                print(f"Breaked at {str(epoch)}")
                break
            
    def get_model(self):
        return self.model    
    
    def plot_loss(self):
        plt.plot(range(len(self.train_loss)), self.train_loss, 'g', label='Training loss')
        plt.plot(range(len(self.val_loss)), self.val_loss, 'y', label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        
    def evaluate(self):
        self.model = self.model.cpu()
        self.graph = self.graph.cpu()
        y_dict = Evaluation.create_y_dict(self.model.predict(self.graph, self.graph.test_mask), self.model.predict_proba(self.graph, self.graph.test_mask) , self.graph.y[self.graph.test_mask])
        y_dict_gw = Evaluation.create_y_dict(self.model.predict(self.graph, self.graph.test_gw_mask), self.model.predict_proba(self.graph, self.graph.test_gw_mask) , self.graph.y[self.graph.test_gw_mask])
        eval_df = Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw)
        print(eval_df)
        self.model = self.model.to(device)


In [ ]:
model_wrapper = ModelWrapper(graph)
model_wrapper.train()

In [ ]:
import time
from dataAnalysis.Metrics import Evaluation
train_times = []
eval_int_times = []
eval_ext_times = []
models= []
dfs = []
for i in range(100-32):
    graph = graph.to(device)
    model_wrapper = ModelWrapper(graph)
    
    start = time.time()
    model_wrapper.train()
    print(start -time.time())
    train_times.append(start -time.time())
    
    model = model_wrapper.get_model()
    graph = graph.cpu()
    model = model.cpu()
    models.append(model)
      
    start = time.time()
    y_dict = Evaluation.create_y_dict(model.predict(graph, test_mask), model.predict_proba(graph, test_mask) , graph.y[test_mask])
    eval_int_times.append(time.time() - start)
    
    start = time.time()
    y_dict_gw = Evaluation.create_y_dict(model.predict(graph, test_gw_mask), model.predict_proba(graph, test_gw_mask) , graph.y[test_gw_mask])
    eval_ext_times.append(time.time() - start)
    df = Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw)
    print(df)
    dfs.append(df)

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaked at 4684
-4344.16086101532
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.063985  0.864122  0.470665   0.014299  0.887630  0.036553
1  Greifswald  0.048226  0.843416  0.462060   0.009129  0.854992  0.019775


  0%|          | 0/10000 [00:00<?, ?it/s]

Breaked at 7768
-9636.467914104462
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.065910  0.866148  0.471473   0.014750  0.885658  0.036421
1  Greifswald  0.048388  0.847808  0.463423   0.009272  0.850309  0.019303


  0%|          | 0/10000 [00:00<?, ?it/s]

Breaked at 6184
-5854.437874078751
         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.066019  0.868553  0.472237   0.014895  0.889846  0.036650
1  Greifswald  0.047735  0.849346  0.463846   0.009218  0.853642  0.021625


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
model_wrapper.plot_loss()

In [ ]:
model = model_wrapper.get_model()
graph = graph.cpu()
model = model.cpu()

In [ ]:
torch.save(model.state_dict(), "directed_gat_corr.pt")

In [ ]:
from dataAnalysis.Metrics import Evaluation

y_dict = Evaluation.create_y_dict(model.predict(graph, test_mask), model.predict_proba(graph, test_mask) , graph.y[test_mask])
y_dict_gw = Evaluation.create_y_dict(model.predict(graph, test_gw_mask), model.predict_proba(graph, test_gw_mask) , graph.y[test_gw_mask])

In [ ]:
from dataAnalysis.Metrics import Evaluation

Evaluation.plot_confusion_matrix_from_pred(model.predict(graph, test_mask), graph.y[test_mask])
Evaluation.plot_confusion_matrix_from_pred(model.predict(graph, test_gw_mask), graph.y[test_gw_mask])
Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw)

In [ ]:
from IPython.display import Audio
sound_file = './finish_sound.mp3'
Audio(sound_file, autoplay=True)

In [ ]:
##TODO:
# - change weight
# - gradients as features
# - different learning rate and number of layers
# - general architecture- less neurons but linear layer before and after convs? 
#       -adding hidden layers with each others for more stable training (guaranteed only when hidden dim is euqal betwene layers)
# - GATv2
# - MIMIC und Greifswald
# - more features

In [ ]:
import time

for i in range(10000):
    time.sleep(60)
    print("Sleeping")